## Import dependencies

In [0]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import random
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np
import pickle

print(tf.__version__)

## Download and explore the MNIST dataset
The MNIST database contains 60,000 training images and 10,000 testing images of handwritten digits. We will use the dataset to train our digit classification model.

Each image in the MNIST dataset is a 28x28 grayscale image containing a digit from 0 to 9, and a label identifying which digit is in the image.
![MNIST sample](https://github.com/khanhlvg/DigitClassifier/raw/master/images/mnist.png)

In [0]:
df_train=pd.read_csv('train_final.csv',index_col=False)
labels=df_train[['784']]


In [0]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

In [0]:
labels=np.array(labels)
cat=to_categorical(labels,num_classes=13)
print(cat[0])
cat.shape

In [0]:
l=[]
for i in range(47504):
    l.append(np.array(df_train[i:i+1]).reshape(28,28))

l = np.array(l)
l = l/255.0

In [0]:
# Define the model architecture
print(np.array(l).shape)
print(cat.shape)
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(30, (5, 5), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Conv2D(15, (3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(50, activation='relu'),
  keras.layers.Dense(13, activation='softmax'),
])

# Define how to train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Train the digit classification model
model.fit(l, cat, epochs=10, batch_size=200,shuffle=True,verbose=1)


Let's take a closer look at our model structure.

In [0]:
model.summary()

## Convert the Keras model to TensorFlow Lite

Now as we have trained the digit classifer model, we will convert it to TensorFlow Lite format for mobile deployment.

In [0]:
# Convert Keras model to TF Lite format.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

As we will deploy our model to a mobile device, we want our model to be as small and as fast as possible. **Quantization** is a common technique often used in on-device machine learning to shrink ML models. Here we will use 8-bit number to approximate our 32-bit weights, which in turn shrinks the model size by a factor of 4.

See [TensorFlow documentation](https://www.tensorflow.org/lite/performance/post_training_quantization) to learn more about other quantization techniques.

In [0]:
# Re-convert the model to TF Lite using quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

## Download the TensorFlow Lite model

Let's get our model and integrate it into an Android app.

If you see an error when downloading mnist.tflite from Colab, try running this cell again.

In [0]:
# Save the quantized model to file to the Downloads directory
f = open('mnist_final_4.tflite', "wb")
f.write(tflite_float_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('mnist_final_4.tflite')

print('`mnist_final_4.tflite` has been downloaded')